# Finetuning with Trainer and text-classification model validation

In [23]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   colab = True
else:
   colab = False

if colab:
    !pip install transformers[torch]
    !pip install accelerate -U
    !pip install datasets
    !pip install torchinfo
    #ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U

In [24]:
from sklearn.model_selection import train_test_split
import datetime
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import seaborn as sn
import torch
import numbers

from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments, DataCollatorWithPadding, AutoTokenizer, pipeline
from transformers.modelcard import parse_log_history

# Prepare parameters for each run of finetnuning

In [25]:
# from huggingface_hub import notebook_login
# notebook_login()

In [26]:
# models = ['albert', 'bart', 'bert', 'big_bird', 'bigbird_pegasus', 'biogpt', 'bloom', 'camembert', 'canine', 'code_llama', 'convbert', 'ctrl', 'data2vec-text', 'deberta', 'deberta-v2', 'distilbert', 'electra', 'ernie', 'ernie_m', 'esm', 'falcon', 'flaubert', 'fnet', 'funnel', 'gemma', 'gpt-sw3', 'gpt2', 'gpt_bigcode', 'gpt_neo', 'gpt_neox', 'gptj', 'ibert', 'layoutlm', 'layoutlmv2', 'layoutlmv3', 'led', 'lilt', 'llama', 'longformer', 'luke', 'markuplm', 'mbart', 'mega', 'megatron-bert', 'mistral', 'mixtral', 'mobilebert', 'mpnet', 'mpt', 'mra', 'mt5', 'mvp', 'nezha', 'nystromformer', 'open-llama', 'openai-gpt', 'opt', 'perceiver', 'persimmon', 'phi', 'plbart', 'qdqbert', 'qwen2', 'reformer', 'rembert', 'roberta', 'roberta-prelayernorm', 'roc_bert', 'roformer', 'squeezebert', 'stablelm', 'starcoder2', 't5', 'tapas', 'transfo-xl', 'umt5', 'xlm', 'xlm-roberta', 'xlm-roberta-xl', 'xlnet', 'xmod', 'yoso',]
# =============================================
# tested = 'distilbert-base-cased', 'camembert-base', 'ctrl', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased'
# ----------
#too weak pc (cannot be loaded or runs at speeds <1 it/s) = 'albert-xlarge-v1', 't5-11B', 'LongformerForSequenceClassification', 'xlm-roberta-base', 'allenai/longformer-base-4096', 'facebook/bart-large',  flaubert/flaubert_large_cased (stopped at 0.5 epoch), 'HuggingFaceH4/tiny-random-LlamaForSequenceClassification'
# ----------
# need changes to run (check error logs) =  ProsusAI/finbert (despite adding padding proposed in error)
# =============================================
# model_path = 'SamLowe/roberta-base-go_emotions'
# model_path = 'distilbert-base-uncased'
# model_path = 'xlmoberta'

# , 'camembert-base', 'ctrl', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased'

# deprecated: transfo-xl-wt103


#if list of objects is provided list of all combinations of parameters will be created for running
params_tested = {'model': ['roberta-base'],
                 'num_train_epochs': 1,
                 'save_strategy': 'no', # 'epoch'/'no'
                 'per_device_train_batch_size': 8,
                 'per_device_eval_batch_size': 64,
                 'split': (90, 10) #may be tuple of len 2/3, dividing respectively into 2:{train, test} and 3:{train, test, validate}. By default labels are stratified. You may pass "balanced" instead of the last element, number of labels for each class will be     
                 }

save_logs = False

def radio_split_tuple():
    split_s = sum([i for i in params_tested['split'] if isinstance(i, numbers.Integral)])
    params_tested['split'] = tuple([i/split_s if isinstance(i, numbers.Integral) else i for i in params_tested['split']])
     


params_tested={i:[q] if type(q) is not list else q for (i,q) in params_tested.items()}
keys = list(params_tested.keys())
combinations = list(itertools.product(*params_tested.values()))
result = [{keys[i]: combination[i] for i in range(len(keys))} for combination in combinations]
print(*result, sep='\n')

train_params_looped = result

{'model': 'roberta-base', 'num_train_epochs': 1, 'save_strategy': 'no', 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 64, 'split': (90, 10)}


## Functions

In [30]:
def df_to_ds(df, train_params_instance):
    temp_split = train_params_instance['split']
    
    target_map = {k:i for i,k in enumerate(df['label'].unique())}
    reversed_target_map = {v:k for k, v in target_map.items()}
    
    df['label'] = df['label'].map(target_map)
    
    if temp_split[-1]=='balanced' or len(temp_split)==2:
        if temp_split[-1]=='balanced':
            split = [pd.Series(name='sentence'), pd.Series(name='sentence'), pd.Series(name='label'), pd.Series(name='label')]
            n_for_training = df.groupby('label').count()['Source'].min()
            train_percentage = df.groupby('label').count()['Source'].apply(lambda x: (temp_split[0]*n_for_training)/x).to_dict()
            
            for label_name, group in df.groupby('label'):
                split_t = train_test_split(group['sentence'], group['label'], test_size=1-train_percentage[label_name], random_state=42, shuffle=True)
                split = [pd.concat([i[0],i[1]], axis=0) for i in zip(split, split_t)]
        else:
            split = train_test_split(df['sentence'], df['label'], stratify=df['label'], test_size=temp_split[1], random_state=42, shuffle=True)
        ds = DatasetDict()
        ds['train'] = Dataset.from_pandas(pd.concat([split[0], split[2]], axis=1))
        ds['test'] = Dataset.from_pandas(pd.concat([split[1], split[3]], axis=1))
    elif len(temp_split)==3:
        non_train_size = 1-temp_split[0]
        split = train_test_split(df['sentence'], df['label'], stratify=df['label'], test_size=non_train_size, random_state=42, shuffle=True)
        split2 = train_test_split(df['sentence'], df['label'], stratify=df['label'], test_size=temp_split[2]/non_train_size, random_state=42, shuffle=True)
        ds = DatasetDict()
        ds['train'] = Dataset.from_pandas(pd.concat([split[0], split[2]], axis=1))
        ds['test'] = Dataset.from_pandas(pd.concat([split2[0], split2[2]], axis=1))
        ds['validate'] = Dataset.from_pandas(pd.concat([split2[1], split2[3]], axis=1))
    return ds, target_map, reversed_target_map

def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average='macro')
  return {'accuracy': acc, 'f1': f1}

def save_logs_from_training_run(trainer, train_params, timestamp, trained_model_path, colab, target_map):
    log_history = parse_log_history(trainer.state.log_history)
    log_df = pd.DataFrame(log_history[1])
    log_df.insert(0, 'model', train_params['model'])
    log_df.insert(0, 'timestamp', timestamp)
    log_df['model_path'] = trained_model_path
    log_df['samples_per_s'] = log_history[0]['train_samples_per_second']
    log_df['steps_per_s'] = log_history[0]['train_steps_per_second']
    log_df['colab'] = colab
    log_df['per_device_train_batch_size'] = train_params['per_device_train_batch_size']
    log_df['per_device_eval_batch_size'] = train_params['per_device_eval_batch_size']
    log_df['split'] = str(train_params['split'])
    log_df['target_map'] = str(target_map)

    float_cols = log_df.select_dtypes(include='float64')
    log_df[float_cols.columns] = float_cols.apply(lambda x: round(x, 3))
    log_df.to_csv('output/training_logs.csv', mode='a', header= not os.path.isfile('output/training_logs.csv'), index=False)
    
def finetune(train_params, ds, target_map, save_logs):
    params_passed = {k: train_params[k] for k in train_params if k not in ['model', 'split']}
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    trained_model_path = f'output/models/{train_params['model']}_{timestamp}'
    try:
        def tokenize_fn(batch):
            return tokenizer(batch['sentence'], truncation=True)
        
        #Tokenize dataset
        tokenizer = AutoTokenizer.from_pretrained(train_params['model'], trust_remote_code=True)
        # tokenizer.pad_token = tokenizer.eos_token
        # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        tokenized_datasets = ds.map(tokenize_fn, batched=True)
        
        #Change labels
        config = AutoConfig.from_pretrained(train_params['model'], trust_remote_code=True)
        config.vocab_size = tokenizer.vocab_size
        config.id2label = reversed_target_map
        config.label2id = target_map
        
        #Load model
        model = AutoModelForSequenceClassification.from_pretrained(
            train_params['model'], config=config, ignore_mismatched_sizes=True, trust_remote_code=True)
    
        training_args = TrainingArguments(
          output_dir=f'{trained_model_path}/checkpoints',
          evaluation_strategy='epoch',
          logging_strategy='epoch',
          use_cpu = False,
          **params_passed
        )
    
        trainer = Trainer(
            model,
            training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )
        
        trainer.train()
        if save_logs:
            save_logs_from_training_run(trainer, train_params, timestamp, trained_model_path, colab, target_map)

    except Exception as e:
        print(f'Error: {e}')
        if save_logs:
            err_df = pd.DataFrame([[trained_model_path, str(e)]])
            err_df.to_csv('output/error_logs.csv', mode='a', index=False)
        
        
def to_binary_classification(x):
    if x in ['Positive', 'Negative']:
        return 'Pathos'
    else:
        return x

## Run

In [31]:
# Load PolarIs-Pathos to df
# source: https://github.com/kaatkaa/PolarIs-Corpora/blob/main/PolarIs-Full-Corpus/PolarIs-Full-Corpus-Pathos/PolarIs-Pathos.xlsx
if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  data_path = 'drive/MyDrive/master_lm/PolarIs-Pathos.xlsx'
else:
  data_path = 'data/PolarIs-Pathos.xlsx'

df = pd.read_excel(data_path)
df['label'] = df[['No_pathos', 'Positive', 'Negative']].idxmax(axis=1)
df['label'] = df['label'].apply(lambda x: to_binary_classification(x))
df = df.rename(columns={'Sentence':'sentence'})

In [32]:
#run looped 
for train_params in train_params_looped[:1]:
    ds, target_map, reversed_target_map, = df_to_ds(df, train_params)    
    finetune(train_params, ds, target_map,save_logs)

Map:   0%|          | 0/15578 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Jakub\DataspellProjects\EQILLM\venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.605900,0.502758,0.800000,0.687500
